In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
import time
from tqdm import tqdm

/Users/longxiangzhang/opt/miniconda3/envs/env1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Information
- We will do a few preliminary exercises and also build a character level MLP language model.
- This model will be similar to the model we did in class, except that we will have characters as tokens, not words.
- You will need a conda environment for this, here is general information on this.
 - https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html
 - PyTorch: https://anaconda.org/pytorch/pytorch
 
In the code below, FILL-IN the code necessary in the hint string provided.

### Preliminary exercises
- Please fill in the cells below with the asked for data.

In [2]:
torch.manual_seed(1)

In [3]:
# Create an embedding layer for a vocabulary of size 10 and the word vectors are each of dimension 5.
e = " FILL_IN "

# Extract the embedding for the word whose token index is 3. What is the shape of this vector?
v = " FILL_IN "

# Extract the weight matrix from the layer e.
# Create a linear layer (with no bias) of size 10 by 5 and set it's data to the embedding matrix.
l = " FILL_IN "
l.weight = " FILL_IN "

# Insert inside of the assert below some sort of equality check between l.weight and e.weight; it should pass to true.
# Hint: look up torch.all() and torch.eq()
assert(" FILL_IN ")

In [6]:
# Create a batch of size 2 with entries [0, 1, 2] and [2, 3, 4] in the data batch.
x = " FILL_IN "

In [8]:
# What is the dimesion of this batch ran through the embeding layer?
assert(e(x).shape == " FILL_IN ")

### Constants and configs used below.

In [4]:
DEVICE = "cpu"
LR = 4.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MARKER = '.'
# N-gram level; P(w_t | w_{t-1}, ..., w_{t-n+1}).
# We use 3 words to predict the next word.
n = 4
# Hidden layer dimension.
h = 20
# Word embedding dimension.
m = 20

### Get the dataset and the tokenizer.

In [8]:
class CharDataset(Dataset):
    def __init__(self, words, chars):
        self.words = words
        self.chars = chars
        # Inverse dictionaries mapping char tokens to unique ids and the reverse.
        # Tokens in this case are the unique chars we passed in above.
        # Each token should be mappend to a unique integer and MARKER should have token 0.
        # For example, stoi should be like {'.' -> 0, 'a' -> 1, 'b' -> 2} if I pass in chars = '.ab'.
        self.stoi = dict(zip([char for char in self.chars], [i for i in range(len(self.chars))]))
        self.itos = dict(zip([value for value in self.stoi.values()], [key for key in self.stoi])) # Inverse mapping.

    def __len__(self):
        # Number of words.
        return len(self.words)

    def contains(self, word):
        # Check if word is in self.words and return True/False if it is, is not.
        return word in self.words

    def get_vocab_size(self):
        # Return the vocabulary size.
        return len(self.chars)

    def encode(self, word):
        # Express this word as a list of int ids. For example, maybe ".abc" -> [0, 1, 2, 3].
        # This assumes 'a' -> 1, etc.
        return [self.stoi[char] for char in word]
    
    def decode(self, tokens):
        # For a set of tokens, return back the string.
        # For example, maybe [1, 1, 2] -> "aac"
        return "".join([self.itos[idx] for idx in tokens])

    def __getitem__(self, idx):
        # This is used so we can loop over the data.
        word = self.words[idx]
        return self.encode(word)

In [3]:
print(set("abcc"))

{'a', 'c', 'b'}


In [6]:
def create_datasets(window, input_file = 'names.txt'):
    """
    This takes a file of words and separates all the words.
    It then gets all the characters present in the universe of words and then ouputs the statistics. 
    """
    with open(input_file, 'r') as f:
        data = f.read()
    # Split the file by new lines. You should get a list of names.
    words = data.split("\n")
    words = [word.strip() for word in words] # This gets rid of any trailing and starting white spaces.
    words = list(filter(None, words)) # Filter out all the empty words.
    
    chars = list(set("".join(words))) # This gets the universe of all characters.
    
    # Will force chars to have MARKER having index 0.
    chars= [MARKER] + chars
    
    # Pad each word with a context window of size n-1.
    # Why? a word like "abc" should becomes "..abc.." if the window is size 3.
    # This is some we can get pair of (x, y) data like this: ".." -> "a", ".a" -> "b", "ab" -> "c", "bc" -> ".", "c." -> "."
    # I.e. this allows us to know that "a" is a start character.
    # So you should get something like ["ab", "c"] -> ["..ab..", "..c.."], for example.
    words = ["."*(window - 1) + word + "."*(window - 1) for word in words]
            
    print(f"The number of examples in the dataset: {len(words)}")
    print(f"The number of unique characters in the vocabulary: {len(chars)}")
    print(f"The vocabulary we have is: {''.join(chars)}")

    # Partition the input data into a training, validation, and the test set.
    out_of_sample_set_size = min(2000, int(len(words) * 0.1)) # We use 10% of the training set, or up to 2000 examples.
    test_set_size = 1500
    
    # First, get a random permutation of randomly permute of size len(words).
    # Then, convert this to a list. 
    # This index list is used below to get the train, validation, and test sets.
    rp = torch.randperm(len(words)).tolist()
    
    # Get train, validation, and test set.
    train_words = [words[i] for i in rp[:-out_of_sample_set_size]]
    validation_words = [words[i] for i in rp[-out_of_sample_set_size:-test_set_size]]
    test_words = [words[i] for i in rp[-test_set_size:]]    
    
    print(f"We've split up the dataset into {len(train_words)}, {len(validation_words)}, {len(test_words)} training, validation, and test examples")

    # But the data in the data set objects.
    train_dataset = CharDataset(train_words, chars)
    validation_dataset = CharDataset(validation_words, chars)
    test_dataset = CharDataset(test_words, chars)

    return train_dataset, validation_dataset, test_dataset

In [9]:
train_dataset, validation_dataset, test_dataset = create_datasets(n)

The number of examples in the dataset: 32033
The number of unique characters in the vocabulary: 27
The vocabulary we have is: .lfijgxqkhrpuszbwecdnmyotva
We've split up the dataset into 30033, 500, 1500 training, validation, and test examples


## Explore the data

In [10]:
# Get the first word in "train_dataset"
train_dataset[0]

[0, 0, 0, 24, 14, 3, 11, 23, 10, 26, 9, 0, 0, 0]

In [11]:
# Get the stoi map of train_dataset. How many keys does it have?
train_dataset.stoi

{'.': 0,
 'l': 1,
 'f': 2,
 'i': 3,
 'j': 4,
 'g': 5,
 'x': 6,
 'q': 7,
 'k': 8,
 'h': 9,
 'r': 10,
 'p': 11,
 'u': 12,
 's': 13,
 'z': 14,
 'b': 15,
 'w': 16,
 'e': 17,
 'c': 18,
 'd': 19,
 'n': 20,
 'm': 21,
 'y': 22,
 'o': 23,
 't': 24,
 'v': 25,
 'a': 26}

'abc'

### Get the dataloader

In [15]:
def create_dataloader(dataset, window):
    x_list = []
    y_list = []
    # For ech word.
    for i, word in enumerate(dataset):
        # Grab a context of size window and window-1 characters will be in x, 1 will be in y.
        for j, _ in enumerate(word):
            # If there is no widow of size window left, break.
            if j + window > len(word) - 1:
                break
            word_window = word[j: j+window]
            x, y = word_window[:-1], word_window[-1]
            x_list.append(x)
            y_list.append(y)
            
    return DataLoader(
        TensorDataset(torch.tensor(x_list), torch.tensor(y_list)),
        BATCH_SIZE,
        shuffle=True
    )

In [16]:
train_dataloader = create_dataloader(train_dataset, n)
validation_dataloader = create_dataloader(validation_dataset, n)
test_dataloader = create_dataloader(test_dataset, n)

### Set up the model
- Identical to lecture. Please look over that!

In [27]:
# One of the first Neural language models!
class CharacterNeuralLanguageModel(nn.Module):
    def __init__(self, V, m, h, n):
        super(CharacterNeuralLanguageModel, self).__init__()
        
        # Vocabulary size.
        self.V = V
        
        # Embedding dimension, per word.
        self.m = m
        
        # Hidden dimension.
        self.h = h
        
        # N in "N-gram"
        self.n = n
        
        # Can you change all this stuff to use nn.Linear?
        # Ca also use nn.Parameter(torch.zeros(V, m)) for self.C but then we need one-hot and this is slow.
        self.C = nn.Embedding(V, m)
        self.H = nn.Parameter(torch.zeros(m * (n-1), h))
        self.W = nn.Parameter(torch.zeros(m * (n-1), V))
        self.U = nn.Parameter(torch.zeros(h,V))
        
        self.b = nn.Parameter(torch.zeros(V))
        self.d = nn.Parameter(torch.zeros(h))
        
        self.init_weights()

    def init_weights(self):
        # Intitialize C, H, W, U in a nice way. Use xavier initialization for the weights.
        # On a first run, just pass.
        nn.init.xavier_uniform_(self.C.weight)
        nn.init.xavier_uniform_(self.H)
        nn.init.xavier_uniform_(self.W)
        nn.init.xavier_uniform_(self.U)
        pass # Replace this pass with something else.
        
    def forward(self, x):
        
        # x is of dimenson N = batch size X n-1
        
        # N X (n-1) X m 
        x = self.C(x)
        
        # N
        N = x.shape[0]
        
        # N X (n-1) * m
        x = x.view(N, -1)
    
        # N X V
        y = self.b + torch.matmul(x, self.W) + torch.matmul(nn.Tanh()(self.d + torch.matmul(x, self.H)), self.U)
        
        return y

### Set up the model.

In [30]:
# Identical to lecture.
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
model = CharacterNeuralLanguageModel(
    train_dataset.get_vocab_size(), m, h, n
).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

In [ ]:
# How many parameters does the neural network have?
# Hint: look up model.named_parameters and the method "nelement" on a tensor.
# See also the XOR notebook where we count the gradients that are 0.
# There, we loop over the parameters.
number_parameters = model.named_parameters

In [32]:
model.parameters

<bound method Module.parameters of CharacterNeuralLanguageModel(
  (C): Embedding(27, 20)
)>

### Train the model.

In [33]:
def calculate_perplexity(total_loss, total_batches):
    return torch.exp(torch.tensor(total_loss / total_batches)).item()

In [34]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 500

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        
        logits = model(x)
                        
        # Get the loss.
        loss = criterion(input=logits, target=y.squeeze(-1))

        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients so they don't explode. Look at how this is done in lecture.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # Do an optimization step.
        optimizer.step()
        
        total_loss += loss.item()
        total_batches += 1
                
        if idx % log_interval == 0 and idx > 0:
            perplexity = calculate_perplexity(total_loss,  total_batches)
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(dataloader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [35]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(dataloader):
            logits = model(x)
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, calculate_perplexity(total_loss,  total_batches)

In [36]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = evaluate(validation_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = evaluate(test_dataloader, model, criterion)
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))

987it [00:00, 2689.29it/s]

| epoch   1 |   500/15251 batches | perplexity    9.621 | loss    2.264 
| epoch   1 |  1000/15251 batches | perplexity    8.731 | loss    2.167 


1862it [00:00, 2858.55it/s]

| epoch   1 |  1500/15251 batches | perplexity    8.481 | loss    2.138 
| epoch   1 |  2000/15251 batches | perplexity    8.297 | loss    2.116 


3075it [00:01, 2999.45it/s]

...

| epoch   1 | 13500/15251 batches | perplexity    7.834 | loss    2.059 
| epoch   1 | 14000/15251 batches | perplexity    7.791 | loss    2.053 


14817it [00:05, 2951.83it/s]

| epoch   1 | 14500/15251 batches | perplexity    7.670 | loss    2.037 
| epoch   1 | 15000/15251 batches | perplexity    7.942 | loss    2.072 


15250it [00:05, 2911.82it/s]


ValueError: Expected input batch_size (1) to match target batch_size (0).

Hint: For the above, you should see your loss around 2.0 and going down. Similarly to perplexity which should be aroud 7 to 8.

## Generate some text.

In [59]:
def generate_word(model, dataset, window):
    generated_word = []
    # Set the context to a window-1 length array having just the MARKER character's token_id.
    context = " FILL_IN "
    
    while True:
        logits = model(torch.tensor(context).view(1, -1))
        
        # Get the probabilities from the logits.
        # Hint: softmax!
        probs = " FILL_IN "
        
        # Get 1 sample from a multinomial having the above probabilities.
        token_id = torch.multinomial(" FILL_IN ").item()
        
        # Append the token_id to the generated word.
        " FILL_IN "
        
        # Move the context over 1, drop the first (oldest) token and apped the new one above.
        # The size of the resulting context should be the same.
        # For exaple, if it was "[0, 1, 2]" and you generated 4, it should now be [1, 2, 4].
        context = " FILL_IN "
        
        if token_id == 0:
            # If you generate token_id = 0, i.e. '.', break out.
            " FILL_IN "
    # Return and decode the generated word to a string.        
    return ''.join(dataset.decode(generated_word))

In [61]:
torch.manual_seed(1)
for _ in range(50):
    print(generate_word(model, train_dataset, n))

aha.
ele.
lia.
aldi.
jarorsse.
dez.
bemartti.
rielci.
revy.
madlais.
hoanda.
dacelia.
kalaliey.
chis.
mayas.
tya.
jon.
ama.
tze.
karies.
jos.
ahkl.
bamaka.
anyaamiush.
kazerher.
jami.
nnek.
maremellen.
toquyla.
nzygu.
enyl.
yanstram.
ahazoriexsunya.
sermontiroonn.
eifiah.
rosi.
rouivan.
ynn.
ahdityn.
jassavoli.
wun.
jayvarante.
nor.
ilyn.
marri.
allifare.
kalyi.
daslenshanna.
daniellaenimanaililah.
cyle.
